In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import get_raw_data
import get_preprocessing_data
import analysis_method
import statsmodels.regression.linear_model as sm

from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

pv_list = ["0102", "1701", "1702", "0111", "0330", "0331", "0332", "0333", "0101", "0107", "0110", "5501"]
#["0102"(가구주와의 관계), "1701"(근로소득 유무), "1702"(총 연간근로소득), "0111"(학력졸업여부), "0330"(업종8차),
# "0331"(업종9차), "0332"(직종8차), "0333"(직종9차), "0101"(성별), "0107"(만나이), "0110"(학력학교), "5501"(혼인상태)]
hv_list = ["0150", "1406", "2102", "2134", "2136", " 2138", "2140", "2142", "2152", "2153", "2155",
           "2156", "2157", "2158", "2159", "2160", "2182", "2183", "2184", "2185", "2186", "2187",
           "2189", "2190", "2191"]
income_code_list = ["2102", "2134", "2136", " 2138", "2140", "2142", "2152", "2153", "2155", "2156",
                    "2157", "2158", "2159", "2160", "2182", "2183", "2184", "2185", "2186", "2187",
                    "2189", "2190", "2191"]

cpi_data = pd.read_excel("C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\CPI.xls").loc[0, :]
cpi_data_index = pd.read_excel("C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\CPI.xls", dtype="str").loc[14, :].values[1:]
cpi_data = pd.Series(cpi_data.values[1:], dtype="float")
cpi_data.name = "CPI"
cpi_data.index = cpi_data_index

industry_8 = pd.read_excel("C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\국내데이터_산업분류.xlsx", sheet_name="8차")
industry_large = pd.read_excel("C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\국내데이터_산업분류.xlsx", sheet_name="대분류")

industry_return = pd.read_excel("C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\oversea_return.xlsx", index_col=0)

data_num = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21"]

cpi_multiple = get_preprocessing_data.get_cpi_multi(cpi_data, data_num)

data_dict = {}
for i in data_num:
    h_path = "C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\h_data\\h_" + i + ".pickle"
    p_path = "C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\p_data\\p_" + i + ".pickle"

    h_data = pd.read_pickle(h_path)
    p_data = pd.read_pickle(p_path)

    raw_hdata = get_raw_data.get_hv_data(h_data, hv_list, i)
    raw_pdata = get_raw_data.get_pv_data(p_data, pv_list, i)
    
    fin_p_data = get_preprocessing_data.get_fin_personal(raw_pdata, industry_8, industry_large)
    income_data = get_preprocessing_data.get_income_data(fin_p_data.index, raw_hdata)

    fin_x_value = get_preprocessing_data.get_final_xvalue(raw_hdata, fin_p_data)
    fin_y_value = get_preprocessing_data.get_final_yvalue(income_data, cpi_multiple[i])

    finally_preprocessed_data = get_preprocessing_data.get_total_data(fin_y_value, fin_x_value)

    data_dict[i] = finally_preprocessed_data

C:\Users\JeayongYu\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
data_df = pd.DataFrame()
for i in data_dict.keys():
    data_df = pd.concat([data_df, data_dict[i]])

_______________________________


### log(Y*)의 계산

In [3]:
temp_edu = data_df.groupby(["0110"]).get_group(1)
temp_edu = get_preprocessing_data.get_agecut_data(temp_edu, 22)
x_mat = analysis_method.get_reg_x(temp_edu).iloc[:, :3]
fer_results = analysis_method.fe_regression_sol(temp_edu, False)

poly_x_mat = analysis_method.polynomial_matrix(temp_edu["0107"], 3) .iloc[:, 1:]
poly_x_mat.index = x_mat.index
fitting_result = analysis_method.fitting_polynomial(fer_results, 3, False)

In [4]:
fer_results[0][:3]

intercept     7.507209
marriage      0.319783
family_num    0.058957
dtype: float64

In [5]:
fitting_result[1]

intercept    -0.741728
age^1/10^0    0.012367
age^2/10^1    0.006378
age^3/10^2   -0.000899
dtype: float64

In [6]:
fitting_paramters = pd.concat([fer_results[0][:3], fitting_result[1]], axis=1).sum(axis=1)
fitting_paramters

intercept     6.765481
marriage      0.319783
family_num    0.058957
age^1/10^0    0.012367
age^2/10^1    0.006378
age^3/10^2   -0.000899
dtype: float64

In [7]:
real_data = pd.concat([x_mat, poly_x_mat], axis=1)
real_data

,intercept,marriage,family_num,age^1/10^0,age^2/10^1,age^3/10^2
hhid,,,,,,
21.0,1,1.0,3.0,25.0,62.5,156.25
25.0,1,1.0,4.0,37.0,136.9,506.53
32.0,1,1.0,4.0,59.0,348.1,2053.79
33.0,1,1.0,4.0,42.0,176.4,740.88
40.0,1,1.0,5.0,47.0,220.9,1038.23
...,...,...,...,...,...,...
110085.0,1,1.0,4.0,35.0,122.5,428.75
110102.0,1,1.0,2.0,30.0,90.0,270.00
110122.0,1,1.0,4.0,37.0,136.9,506.53


In [8]:
temp_edu["labor income"]

hhid
21.0        1940.968781
25.0        5822.906343
32.0        2911.453171
33.0        3881.937562
40.0        5822.906343
               ...     
110085.0    6140.000000
110102.0    3500.000000
110122.0    6678.000000
110235.0    6560.000000
110250.0    9700.000000
Name: labor income, Length: 23510, dtype: float64

In [9]:
real_data @ fitting_paramters

hhid
21.0        7.829511
25.0        8.196628
32.0        8.425393
33.0        8.299808
40.0        8.437226
              ...   
110085.0    8.149944
110102.0    7.905564
110122.0    8.196628
110235.0    8.221976
110250.0    8.055704
Length: 23510, dtype: float64

_______________________________


### 3rd-order polynomials vision(각 parameter의 계수를 한번 추정하여 rid계산하는 경우)

In [10]:
fitting_paramters

intercept     6.765481
marriage      0.319783
family_num    0.058957
age^1/10^0    0.012367
age^2/10^1    0.006378
age^3/10^2   -0.000899
dtype: float64

----------------------------------------
### shock calculation

In [11]:
group_sigma_df = pd.DataFrame()
group_pvalue_df = pd.DataFrame()
group_rid_df = pd.DataFrame()

start_list = [22, 24, 26]
for i in range(data_df["0110"].max()):
    df_y_star = pd.DataFrame()

    temp_edu = data_df.groupby(["0110"]).get_group(i+1)
    temp_edu = get_preprocessing_data.get_agecut_data(temp_edu, start_list[i])
    
    fer_results = analysis_method.fe_regression_sol(temp_edu, False)
    fitting_result = analysis_method.fitting_polynomial(fer_results, 3, False)

    fitting_paramters = pd.concat([fer_results[0][:3], fitting_result[1]], axis=1).sum(axis=1)
    
    for j in data_num:
        temp_edu_split = data_dict[j].groupby(["0110"]).get_group(i+1)
        temp_edu_split = get_preprocessing_data.get_agecut_data(temp_edu_split, start_list[i])
        
        log_y = np.log(temp_edu_split["labor income"])
        
        x_mat = analysis_method.get_reg_x(temp_edu_split).iloc[:, :3]
        poly_x_mat = analysis_method.polynomial_matrix(temp_edu_split["0107"], 3) .iloc[:, 1:]
        poly_x_mat.index = x_mat.index
        
        temp_real_data = pd.concat([x_mat, poly_x_mat], axis=1)
    
        f_hat = temp_real_data @ fitting_paramters
        
        log_y_star = log_y - f_hat
        df_y_star = pd.concat([df_y_star, log_y_star], axis=1)

    df_y_star.columns = data_num
    
    temp_varrid_list = []
    temp_rid_list = []
    for k in range(len(data_num)-1):
        temp_rid = df_y_star.iloc[:, :-(k+1)].values - df_y_star.iloc[:, k+1:].values
        temp_rid_list.append(temp_rid)
        
        temp_varrid_list.append(np.nanvar(temp_rid))
        
    var_rid_series = pd.Series(temp_varrid_list)
    
    rid_series = (pd.DataFrame(temp_rid_list[0], columns = data_num[1:]) * -1).mean()
    group_rid_df = pd.concat([group_rid_df, rid_series], axis=1)
    
    var_rid_x = np.ones([len(var_rid_series), 2]) * 2
    var_rid_x[:, 0] = var_rid_series.index.values + 1
    var_rid_y = var_rid_series.values
    
    sol = sm.OLS(var_rid_y, var_rid_x)
    sol_result = sol.fit()
    
    sigma_df = pd.DataFrame(sol_result.params, index=["permanent_shock", "idiosyncratic_temporary_shock"], columns=["sigma"])
    pvalue_df = pd.DataFrame(np.round(sol_result.pvalues, 3), index=["permanent_shock(p-value)", "idiosyncratic_temporary_shock(p-value)"], columns=["p-value"])
    
    group_sigma_df = pd.concat([group_sigma_df, sigma_df], axis=1)
    group_pvalue_df = pd.concat([group_pvalue_df, pvalue_df], axis=1)

group_rid_df.columns = ["high_school", "college", "university"]
group_sigma_df.columns = ["high_school", "college", "university"]
group_pvalue_df.columns = ["high_school", "college", "university"]
group_sigma_df = pd.concat([group_sigma_df, group_pvalue_df]).loc[["idiosyncratic_temporary_shock", "permanent_shock", "idiosyncratic_temporary_shock(p-value)", "permanent_shock(p-value)"],:]
group_sigma_df

,high_school,college,university
idiosyncratic_temporary_shock,0.174855,0.145682,0.141262
permanent_shock,0.005461,0.010166,0.005379
idiosyncratic_temporary_shock(p-value),0.000000,0.000000,0.000000
permanent_shock(p-value),0.001000,0.000000,0.009000


----------------------------------------
### Beta & Correlation

In [12]:
ksi = group_rid_df
ksi

,high_school,college,university
02,0.011292,-0.035928,-0.017210
03,0.093463,0.068189,0.046215
04,-0.014766,-0.026539,0.134206
05,0.195853,0.314452,0.109214
06,0.054299,-0.018083,0.038344
07,0.071300,0.107663,0.040838
08,0.030607,0.090565,0.078086
09,0.104878,0.110067,0.075320
10,0.040013,0.111453,0.076239
11,0.032881,-0.001108,0.062704


In [13]:
industry_return

,다우 산업,나스닥 종합,S&P 500,대한민국 - KOSPI,니케이 225,대만 가권,독일 DAX,영국 FTSE 100,러셀 3000,항셍,국고채 총수익 지수,국고채 콜 재투자 지수,국고채 제로 재투자 지수,KRW Cash 총수익 지수
Symbol Name,,,,,,,,,,,,,,
1,17.30,16.75,23.90,-43.07,-23.27,17.65,45.31,22.51,23.42,-22.48,3.901538,3.796154,3.634615,2.407143
2,20.15,44.26,30.86,49.93,-7.39,-20.03,21.96,17.02,25.48,-1.67,3.901538,3.796154,3.634615,2.407143
3,24.39,82.18,18.95,82.78,37.01,30.35,36.14,16.01,19.40,66.45,3.901538,3.796154,3.634615,2.407143
4,-6.25,-37.18,-9.82,-49.11,-24.63,-42.56,-3.71,-9.38,-8.35,-8.98,3.901538,3.796154,3.634615,2.407143
5,-6.73,-22.30,-12.98,37.47,-24.41,12.53,-19.01,-15.76,-12.72,-22.74,3.901538,3.796154,3.634615,2.407143
6,-17.80,-32.59,-24.26,-9.54,-18.63,-17.42,-43.94,-25.60,-23.74,-18.89,3.901538,3.796154,3.634615,2.407143
7,25.11,50.04,26.18,29.19,24.45,31.61,37.08,14.61,28.82,35.10,3.901538,3.796154,3.634615,2.407143
8,3.60,8.38,9.36,10.51,7.61,3.99,7.34,7.82,10.25,13.07,3.901538,3.796154,3.634615,2.407143
9,-0.41,1.89,3.38,55.99,43.60,8.00,28.51,16.98,4.71,5.46,-0.770000,-0.620000,-0.760000,2.407143


In [14]:
industry_return.index = data_num
industry_ret = industry_return.copy() / 100


beta_dict = {}
for i in ksi.columns:
    beta_list = []
    pvalue_list = []
    eta_list = []
        
    temp_excess_ret = (industry_ret - 0.02)
    for k in industry_ret.columns:
        temp_eta = temp_excess_ret[k] - np.mean(temp_excess_ret)[k]
        eta_list.append(temp_eta)
            
        temp_beta_model = sm.OLS(ksi[i], temp_eta.loc[ksi.index])
        temp_beta_results = temp_beta_model.fit()
            
        beta_list.append(temp_beta_results.params[0])
        pvalue_list.append(temp_beta_results.pvalues[0])
        
    eta = pd.DataFrame(eta_list, index=industry_ret.columns).T
    beta = pd.DataFrame(beta_list, index=industry_ret.columns, columns=["Beta"])
    pvalue = pd.DataFrame(pvalue_list, index=industry_ret.columns, columns=["p-value"])
    
    beta_dict[i] = pd.concat([beta, pvalue], axis=1).T

In [15]:
pd.DataFrame.from_dict({(i,j,k): beta_dict[i][k]
                        for i in beta_dict.keys()
                        for k in beta_dict[i].columns}, orient="index")

Beta   p-value
high_school 21 다우 산업            0.063851  0.538116
               나스닥 종합           0.036248  0.489863
               S&P 500          0.036120  0.690088
               대한민국 - KOSPI     0.111661  0.020483
               니케이 225          0.073302  0.259662
               대만 가권            0.098405  0.096572
               독일 DAX           0.024774  0.725591
               영국 FTSE 100      0.052081  0.637191
               러셀 3000          0.043731  0.627951
               항셍               0.050684  0.386805
               국고채 총수익 지수      -0.843611  0.190872
               국고채 콜 재투자 지수    -0.882510  0.196286
               국고채 제로 재투자 지수   -0.896395  0.199126
               KRW Cash 총수익 지수  0.788646  0.829745
college     21 다우 산업           -0.008601  0.948744
               나스닥 종합          -0.009541  0.888093
               S&P 500         -0.030194  0.795366
               대한민국 - KOSPI     0.093722  0.146523
               니케이 225          0.039001  0.645077
               대만 가권            0.087202  0.260781
               독일 DAX           0.000439  0.996144
               영국 FTSE 100     -0.013528  0.924185
               러셀 3000         -0.020204  0.861906
               항셍               0.012188  0.872448
               국고채 총수익 지수      -0.396371  0.638871
               국고채 콜 재투자 지수    -0.409896  0.646602
               국고채 제로 재투자 지수   -0.442018  0.628401
               KRW Cash 총수익 지수  0.373163  0.936856
university  21 다우 산업           -0.033993  0.693240
               나스닥 종합          -0.035515  0.412881
               S&P 500         -0.044267  0.554239
               대한민국 - KOSPI     0.011266  0.792450
               니케이 225          0.002377  0.965400
               대만 가권            0.004623  0.927638
               독일 DAX          -0.007449  0.898772
               영국 FTSE 100     -0.022881  0.802914
               러셀 3000         -0.034093  0.648459
               항셍               0.023715  0.627374
               국고채 총수익 지수      -0.355814  0.512737
               국고채 콜 재투자 지수    -0.361261  0.530207
               국고채 제로 재투자 지수   -0.374105  0.524668
               KRW Cash 총수익 지수  0.363591  0.904786

In [16]:
corr_df = pd.DataFrame()
for i in [0, 1, 2]:
        
    corr_list = []
    for k in eta.columns:
        corr_list.append(np.corrcoef(eta.loc[ksi.index, :][k], ksi[ksi.columns[i]])[0, 1])
        
    temp_corr_df = pd.DataFrame(corr_list, index=eta.columns)
    corr_df = pd.concat([corr_df, temp_corr_df], axis=1)
    
corr_df.columns = ksi.columns
corr_df

,high_school,college,university
다우 산업,0.236669,0.001927,-0.107379
나스닥 종합,0.236592,-0.034212,-0.291275
S&P 500,0.179751,-0.042711,-0.161960
대한민국 - KOSPI,0.632348,0.333802,-0.012220
니케이 225,0.313472,0.091419,-0.052613
대만 가권,0.559065,0.319192,0.062226
독일 DAX,0.195925,0.049623,0.046925
영국 FTSE 100,0.222525,0.013794,-0.014368
러셀 3000,0.205978,-0.020596,-0.115035
항셍,0.229621,0.009172,0.110845


In [17]:
corr_mat_dic = {}
for i in ksi.columns:
    corr_mat_dic[i] = pd.concat([ksi[i], eta], axis=1).dropna().corr()
    
sample_corr_mat = pd.DataFrame.from_dict({(i,k): corr_mat_dic[i][k]
                            for i in corr_mat_dic.keys()
                            for k in corr_mat_dic[i].columns}, orient="index")
sample_corr_mat

high_school     다우 산업    나스닥 종합   S&P 500  \
high_school high_school         1.000000  0.236669  0.236592  0.179751   
            다우 산업               0.236669  1.000000  0.882829  0.964475   
            나스닥 종합              0.236592  0.882829  1.000000  0.879723   
            S&P 500             0.179751  0.964475  0.879723  1.000000   
            대한민국 - KOSPI        0.632348  0.563329  0.732791  0.561052   
            니케이 225             0.313472  0.698366  0.724533  0.727173   
            대만 가권               0.559065  0.674701  0.690628  0.646143   
            독일 DAX              0.195925  0.808059  0.808670  0.851531   
            영국 FTSE 100         0.222525  0.866260  0.819400  0.909939   
            러셀 3000             0.205978  0.965277  0.876583  0.995620   
            항셍                  0.229621  0.760128  0.792528  0.704446   
            국고채 총수익 지수         -0.427067 -0.509092 -0.336330 -0.476106   
            국고채 콜 재투자 지수       -0.422141 -0.503863 -0.335359 -0.472419   
            국고채 제로 재투자 지수      -0.419593 -0.500196 -0.331631 -0.469736   
            KRW Cash 총수익 지수     0.071798 -0.085436 -0.012376  0.012174   
college     college                  NaN  0.001927 -0.034212 -0.042711   
            다우 산업                    NaN  1.000000  0.882829  0.964475   
            나스닥 종합                   NaN  0.882829  1.000000  0.879723   
            S&P 500                  NaN  0.964475  0.879723  1.000000   
            대한민국 - KOSPI             NaN  0.563329  0.732791  0.561052   
            니케이 225                  NaN  0.698366  0.724533  0.727173   
            대만 가권                    NaN  0.674701  0.690628  0.646143   
            독일 DAX                   NaN  0.808059  0.808670  0.851531   
            영국 FTSE 100              NaN  0.866260  0.819400  0.909939   
            러셀 3000                  NaN  0.965277  0.876583  0.995620   
            항셍                       NaN  0.760128  0.792528  0.704446   
            국고채 총수익 지수               NaN -0.509092 -0.336330 -0.476106   
            국고채 콜 재투자 지수             NaN -0.503863 -0.335359 -0.472419   
            국고채 제로 재투자 지수            NaN -0.500196 -0.331631 -0.469736   
            KRW Cash 총수익 지수          NaN -0.085436 -0.012376  0.012174   
university  university               NaN -0.107379 -0.291275 -0.161960   
            다우 산업                    NaN  1.000000  0.882829  0.964475   
            나스닥 종합                   NaN  0.882829  1.000000  0.879723   
            S&P 500                  NaN  0.964475  0.879723  1.000000   
            대한민국 - KOSPI             NaN  0.563329  0.732791  0.561052   
            니케이 225                  NaN  0.698366  0.724533  0.727173   
            대만 가권                    NaN  0.674701  0.690628  0.646143   
            독일 DAX                   NaN  0.808059  0.808670  0.851531   
            영국 FTSE 100              NaN  0.866260  0.819400  0.909939   
            러셀 3000                  NaN  0.965277  0.876583  0.995620   
            항셍                       NaN  0.760128  0.792528  0.704446   
            국고채 총수익 지수               NaN -0.509092 -0.336330 -0.476106   
            국고채 콜 재투자 지수             NaN -0.503863 -0.335359 -0.472419   
            국고채 제로 재투자 지수            NaN -0.500196 -0.331631 -0.469736   
            KRW Cash 총수익 지수          NaN -0.085436 -0.012376  0.012174   

                             대한민국 - KOSPI   니케이 225     대만 가권    독일 DAX  \
high_school high_school          0.632348  0.313472  0.559065  0.195925   
            다우 산업                0.563329  0.698366  0.674701  0.808059   
            나스닥 종합               0.732791  0.724533  0.690628  0.808670   
            S&P 500              0.561052  0.727173  0.646143  0.851531   
            대한민국 - KOSPI         1.000000  0.526866  0.676559  0.627000   
            니케이 225              0.526866  1.000000  0.613626  0.781825   
            대만 가권                0.676559  0.613626  1.000000  0.618660   
            독일 DAX      

In [18]:
corr_mat_dic["university"]

,university,다우 산업,나스닥 종합,S&P 500,대한민국 - KOSPI,니케이 225,대만 가권,독일 DAX,영국 FTSE 100,러셀 3000,항셍,국고채 총수익 지수,국고채 콜 재투자 지수,국고채 제로 재투자 지수,KRW Cash 총수익 지수
university,1.000000,-0.107379,-0.291275,-0.161960,-0.012220,-0.052613,0.062226,0.046925,-0.014368,-0.115035,0.110845,-0.240676,-0.230896,-0.233980,0.044229
다우 산업,-0.107379,1.000000,0.882829,0.964475,0.563329,0.698366,0.674701,0.808059,0.866260,0.965277,0.760128,-0.509092,-0.503863,-0.500196,-0.085436
나스닥 종합,-0.291275,0.882829,1.000000,0.879723,0.732791,0.724533,0.690628,0.808670,0.819400,0.876583,0.792528,-0.336330,-0.335359,-0.331631,-0.012376
S&P 500,-0.161960,0.964475,0.879723,1.000000,0.561052,0.727173,0.646143,0.851531,0.909939,0.995620,0.704446,-0.476106,-0.472419,-0.469736,0.012174
대한민국 - KOSPI,-0.012220,0.563329,0.732791,0.561052,1.000000,0.526866,0.676559,0.627000,0.663290,0.555705,0.657896,-0.476768,-0.474082,-0.471153,-0.086017
니케이 225,-0.052613,0.698366,0.724533,0.727173,0.526866,1.000000,0.613626,0.781825,0.752697,0.754154,0.622554,-0.563398,-0.563677,-0.568442,0.059127
대만 가권,0.062226,0.674701,0.690628,0.646143,0.676559,0.613626,1.000000,0.618660,0.693982,0.685654,0.777838,-0.442126,-0.439915,-0.437005,-0.068288
독일 DAX,0.046925,0.808059,0.808670,0.851531,0.627000,0.781825,0.618660,1.000000,0.916633,0.863210,0.796098,-0.456532,-0.451282,-0.453166,0.055133
영국 FTSE 100,-0.014368,0.866260,0.819400,0.909939,0.663290,0.752697,0.693982,0.916633,1.000000,0.919927,0.755765,-0.560185,-0.554563,-0.554810,-0.075208
러셀 3000,-0.115035,0.965277,0.876583,0.995620,0.555705,0.754154,0.685654,0.863210,0.919927,1.000000,0.726803,-0.487033,-0.483137,-0.480390,0.013161


In [19]:
corr_mat_dic["university"].to_excel("C:\\Users\\JeayongYu\\PycharmProjects\\human_captial\\data\\oversea_corr.xlsx")